# SVF Imagery Generation using RVT

**Load required libraries**

In [ ]:
import os
import time
import rvt.default
import rvt.vis
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor
from itertools import repeat
from svf_gen import *

The next cell identifies the current working directory.

In [ ]:
os.getcwd()

Create variables below for input output folders and a DEM processing listing.

In [ ]:
in_fld = "./OPR"
out_fld = "./SVF"
f = os.listdir(in_fld)
files = []
for i in f:
    if "." in i:
        if i.split(".")[1] == 'tif': files.append(i)

The following cell generates Sky View Factor (SVF) imagery from source digital elevation model (DEM) data with the [Relief Visualization Toolbox Python library](https://github.com/EarthObservation/RVT_py/tree/master).  The processing step also deploys the [multiprocessing](https://docs.python.org/3/library/multiprocessing.html) python library in creating the SVF visualization with default values including 16 directional computation at a radius distance of 10 pixels.

### SVF References

- Kokalj, Ž., Somrak, M. 2019. Why Not a Single Image? Combining Visualizations to Facilitate Fieldwork and On-Screen Mapping. Remote Sensing 11(7): 747. 
- Zakšek, K., Oštir, K., Kokalj, Ž. 2011. Sky-View Factor as a Relief Visualization Technique. Remote Sensing 3: 398-415. 

In [ ]:
start_time = time.time()

default = rvt.default.DefaultValues()

if out_fld != None:
    if not os.path.exists(out_fld):
        os.makedirs(out_fld)
else:
    out_fld = "./SVF"

pool = ProcessPoolExecutor(max_workers=mp.cpu_count())
_ = list(
    pool.map(
        svf_gen,
        files,
        repeat(in_fld),
        repeat(out_fld),
        repeat(default)

    )
)
print(f"\n\n--- {time.time() - start_time} seconds ---")

SVF Image Generation
Dataset|Computer|Tile Count|Time (Seconds)|Time (Minutes)
---|---|---|---|---
NC P3 2015|RDzur-Mac (M1 Max)|11,269|1354.44|22m 34.4s
NC_P3_2015_Coleridge_SE|RDzur-Mac (M1 Max)|30|4.8|

The next cells use [gdal](https://gdal.org) to create raster mosaic for the SVF data.

Create a text listing of the output SVF data.  

In [ ]:
!find ./SVF -name '*.tif' > Reference/NC_P3_2015_TEM_Coleridge_SE_svf.txt

Verify the text file listing.

In [ ]:
!cat Reference/NC_P3_2015_TEM_Coleridge_SE_svf.txt

Below a spatial index of the output SVF can be created with [gdaltindex](https://gdal.org/programs/gdaltindex.html#gdaltindex) using the input text listing.

In [ ]:
!gdaltindex Reference/NC_P3_2015_TEM_Coleridge_SE_svf.shp --optfile Reference/NC_P3_2015_TEM_Coleridge_SE_svf.txt 

The input text listing can also be used as input to develop raster virtual datasets with [gdalbuildvrt](https://gdal.org/programs/gdalbuildvrt.html#gdalbuildvrt).

In [ ]:
!gdalbuildvrt Reference/NC_P3_2015_TEM_Coleridge_SE_svf.vrt -input_file_list Reference/NC_P3_2015_TEM_Coleridge_SE_svf.txt

A physical GeoTIFF (Cloud Optimized - [COG](https://gdal.org/drivers/raster/cog.html#raster-cog)) can be generated for display with [gdalwarp](https://gdal.org/programs/gdalwarp.html) in about ~30 seconds for SVF.

In [ ]:
# %%bash
# START=$(date +%s);
# sleep 1; 
# echo $START
# gdalwarp Reference/NC_P3_2015_TEM_Coleridge_SE_svf.vrt Reference/NC_P3_2015_TEM_Coleridge_SE_svf.tif -of COG -co COMPRESS=LZW -co PREDICTOR=YES -co BLOCKSIZE=128 -co RESAMPLING=CUBIC -co OVERVIEW_PREDICTOR=YES -co OVERVIEW_COUNT=10 -co OVERVIEWS=IGNORE_EXISTING -co BIGTIFF=YES -co ADD_ALPHA=YES -overwrite
# END=$(date +%s);
# echo ----- $((END-START)) seconds -----

!gdalwarp Reference/NC_P3_2015_TEM_Coleridge_SE_svf.vrt Reference/NC_P3_2015_TEM_Coleridge_SE_svf.tif -of COG -co COMPRESS=LZW -co PREDICTOR=YES -co BLOCKSIZE=128 -co RESAMPLING=CUBIC -co OVERVIEW_PREDICTOR=YES -co OVERVIEW_COUNT=10 -co OVERVIEWS=IGNORE_EXISTING -co BIGTIFF=YES -co ADD_ALPHA=YES -overwrite